In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from keras import Sequential
from keras.layers import Dense
from keras import models
from keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
intents = pd.read_csv('intent_data.csv', encoding = 'unicode_escape')
raw = intents
intents

In [ ]:
intents.groupby('intent').size()

In [ ]:
intents["text"] = intents["text"].apply(lambda x: nltk.word_tokenize(x))

In [ ]:
intents["text"]

In [ ]:
def lowercase(tokens):
    tokens_new = []
    for token in tokens:
        t = token.lower()
        tokens_new.append(t)
    return tokens_new

In [ ]:
intents["text"] = intents["text"].apply(lambda x: lowercase(x))

In [ ]:
intents["text"]

In [ ]:
def preprocess(text):
    clean_data = []
    for x in text: 
        new_text = re.sub('[^A-Za-z0-9]+', '', x) # remove punc.         
        if new_text != '':
            clean_data.append(new_text)
    return clean_data

In [ ]:
intents["text"] = intents["text"].apply(lambda x: preprocess(x))
intents["text"]

In [ ]:
def remove_stopwords(text):
    new_token = []
    for t in text:
        if t not in stop_words:
            new_token.append(t)
    return new_token

In [ ]:
intents["text"] = intents["text"].apply(lambda x: remove_stopwords(x))
intents["text"]

In [ ]:
def lemma(text):
    new_token = []
    for t in text:
        word = lemmatizer.lemmatize(t)
        new_token.append(word)
    return new_token

In [ ]:
intents["text"] = intents["text"].apply(lambda x: lemma(x))
intents["text"]

In [ ]:
Y = intents["intent"]
le = LabelEncoder()
le.fit(Y)
print(le.classes_)
Y = le.transform(Y)
Y

In [ ]:
def join(text):
    string = ", ".join(text)
    string = string.replace(",", "")
    return string

In [ ]:
intents["text"] = intents["text"].apply(lambda x: join(x))
intents

In [ ]:
X = intents["text"]
X

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(intents['text'])

In [ ]:
X_Tfidf = Tfidf_vect.transform(X)

In [ ]:
print(Tfidf_vect.vocabulary_)

In [ ]:
print(X_Tfidf)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(X_Tfidf,Y,test_size=0.2)
print("   X         Y")
print(Train_X.shape, Train_Y.shape) # 320 training samples
print(Test_X.shape, Test_Y.shape) # 80 test samples
print(type(Train_X), type(Train_Y))

In [ ]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X, Train_Y)
predictions_NB = Naive.predict(Test_X)
print(accuracy_score(predictions_NB, Test_Y)*100)
print(classification_report(Test_Y, predictions_NB))

In [ ]:
labels = [0,1,2,3,4,5]
cm = confusion_matrix(Test_Y, predictions_NB, labels)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X, Train_Y)
predictions_SVM = SVM.predict(Test_X)
print(accuracy_score(predictions_SVM, Test_Y)*100)
print(classification_report(Test_Y, predictions_SVM))

In [ ]:
LDA = LinearDiscriminantAnalysis()
LDA.fit(Train_X.toarray(), Train_Y)
predictions_LDA = LDA.predict(Test_X)
print(accuracy_score(predictions_LDA, Test_Y)*100)
print(classification_report(Test_Y, predictions_LDA))

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=5)
KNN.fit(Train_X, Train_Y)
predictions_KNN = KNN.predict(Test_X)
print(accuracy_score(predictions_KNN, Test_Y)*100)
print(classification_report(Test_Y, predictions_KNN))